# Submitted By- Diwakar.
Leetcode - https://leetcode.com/diwakarkalra/

LinkedIn - https://www.linkedin.com/in/diwakar-220b19148/

#####     Function "completeOrder" to check if a single warehouse can fullfill the entire order
     @parameter order = order that needs to be fulfilled
     @Paremeter warehouse = warehouse in consideration
     Returns Boolean
 

In [642]:
def completeOrder(order , warehouse):  
    for item in order.keys():
        if item not in warehouse["inventory"].keys() or warehouse["inventory"][item] < order[item]:
            return False
    return True

#####     Function "halfOrder" to check if the current Warehouse can provide some items of the Order
     @parameter remainig_Order = order that needs to be fulfilled
     @Paremeter warehouse = warehouse in consideration
     @parameter start = index of current warehouse
     @parameter end = index of the last Warehouse
     Returns the number of items taken from one warehouse to fulfill the order
     Handles the case if Order cannot be shipped because there is not enough inventory

In [643]:
def halfOrder(remainig_Order, warehouse, start, end) :
    order_filled = {}
    for item in remainig_Order.keys():
        if remainig_Order[item] == 0 :
            continue
        else:
            if item in  warehouse["inventory"].keys() : 
                if warehouse["inventory"][item] >= remainig_Order[item]:   
                    order_filled[item] = remainig_Order[item]
                    remainig_Order[item] = 0
                elif warehouse["inventory"][item] < remainig_Order[item] and warehouse["inventory"][item] > 0:
                    order_filled[item] = warehouse["inventory"][item]
                    remainig_Order[item] -= warehouse["inventory"][item]
            if start == end and remainig_Order[item] > 0:
                return [],remainig_Order
    if not order_filled:
        return {} ,remainig_Order
    else:
        return {warehouse["name"] : order_filled} ,remainig_Order 
        

#####     Function "smallest_Path" to return the best way to ship the order
     @parameter All_paths = All ways by which order can be fulfilled
     Returns the path with least number of warehouses

In [644]:
import math
def smallest_Path(All_paths):
    minimum_length = math.inf
    index = 0
    for i,path in enumerate(All_paths):
        if len(path) < minimum_length:
            minimum_length = len(path)
            index = i
    return All_paths[index]

### Driver Function - inventory_Allocator
    @Parame order = Order to be completed
    @Warehouses = Set of all all warehouse available to satisfy the order
    Returns a path if order can be fulfilled


In [645]:
def inventory_Allocator(order={}, warehouses=[]):
    if not order or type(order) != dict:
        return ("Invalid Order")
    
    if not warehouses or type(warehouses) != list:
        return [] 
    
    All_paths = []
    Original_order = order.copy()
    break_condition = False
    
    for index in range(len(warehouses)) : #Checks all warehouses starting from the nearest 
        if completeOrder(Original_order , warehouses[index]) : 
                return [{warehouses[index]["name"] : Original_order}]
        path_Exists = []
        order_ = order.copy()  #Preserving the state to compute differenet Paths
        
        for sub_index in range( index , len(warehouses)): #checks if we can complete orders using multiple warehouses
            
            for i in range(sub_index+1, len(warehouses)): #checks if at any instant remaing order can be completely 
                                                    # filled by a single warehouse amongst the remaining warehouses
                if completeOrder(order_ , warehouses[i]) :   
                    path_Exists.append({warehouses[i]["name"] : order_})
                    break_condition = True  # Since this is already the best path using current Warehouse[sub_index]
                    break                    # We need not to check any further
                else:
                    continue
                    
            if break_condition == True:
                break_condition = False
                break
                
            items_found , order_ = halfOrder(order_,warehouses[sub_index],sub_index, len(warehouses) -1) 
            
            if items_found == []:
                if not All_paths and sub_index == len(warehouses) -1:
                    return []  #if we have checked all warehouses and we cannot fulfill order
                else:
                    path_Exists = [] #if we cannot fulfill order just using the current warehouse, flush curr_path
                    break
            elif items_found:
                path_Exists.append(items_found)
                        
        if path_Exists and  path_Exists not in All_paths:
            All_paths.append(path_Exists)
            path_Exists = []
    return smallest_Path(All_paths)



## Explanation of Above Solution
The driver function takes into account the order and the warehouses.

A) Assumptions:
    1. Best way to ship order is by using minimum number of warehouses.
    2. The quantities can be postitive or zero but never negative.
    3. The cost of items/inventory in each warehouse is same.

B) Priority Distribution:
    1. Order can be fulfilled by single warehouse.
    2. Order can be fullfilled by more than one warehouse but higher priority to path with lesser number of  
          warehouses.
    3. If there are more than one path with least number of warehouses, higher priority will be given to the path 
        starting with nearest warehouse.
        
C) Workflow
    1. To check if the order can be completed using one warehouse with the help of "completeOrder" function
    2. If not, check if a part of the order can be completed using a warehouse with the help of the "halfOrder"                  function
    3. 'items_found' checks if the current warehouse can help with some items to fulfill the existing order. If                  yes it is added to path_Exists.
    4. path_Exists keeps track if a path is possible with the current warehouse, if yes it is added to All_paths.
    5. All_paths keeps a track of differnet ways to deliver the order.
    6. smallest_Path returns the best way to fullfill the order based upon Assumptions and Priority Distribution             mentioned above.

#### Test Case 1 : if Order and Warehouse are empty

In [646]:
print("T1 : " , inventory_Allocator())

T1 :  Invalid Order


#### Test Case 2 : if Order is blank

In [647]:
print("T2 : " , inventory_Allocator("", {"name" : "aaa" , "inventory" : {"apple" : 1 } }))

T2 :  Invalid Order


#### Test Case 3 : if Warehouses section is blank

In [648]:
print("T3 : " , inventory_Allocator({ "apple" : 2}, ""))

T3 :  []


#### Test Case 4 : Sample Test Case - 1

In [649]:
print("T4 : " , inventory_Allocator({ "apple" : 1}, [{ "name": "owd", "inventory": { "apple": 1 }} ]))

T4 :  [{'owd': {'apple': 1}}]


#### Test Case 5 : Sample Test Case - 2

In [650]:
print("T5 : " , inventory_Allocator({ "apple" : 1}, [{ "name": "owd", "inventory": { "apple": 0 }} ]))

T5 :  []


#### Test Case 6 : Sample Test Case - 3

In [651]:
print("T6 : " , inventory_Allocator({ "apple" : 10 }, 
                                    [{ "name": "owd", "inventory": { "apple": 5 }} , 
                                    { "name": "dm", "inventory": { "apple": 5 } }]))

T6 :  [{'owd': {'apple': 5}}, {'dm': {'apple': 5}}]


#### Test Case 7: To check if Higer Priority is given to Shipment from single warehouse

In [652]:
print("T7 : " , inventory_Allocator({"apple" : 1 , "orange" : 1} , 
                         [{"name" : "aaa" , "inventory" : {"apple" : 1 } } ,
                          {"name" : "bbb" , "inventory" : {"orange" : 1 } } ,
                          {"name" : "ccc" , "inventory" : {"apple" : 1,"orange" : 1}}]))

T7 :  [{'ccc': {'apple': 1, 'orange': 1}}]


#### Test Case 8: To check if there are multiple ways to ship the order using same number of warehouses, we get the result of combinattion starting from the nearest warehouse


In [653]:
print("T8 : " , inventory_Allocator({"apple" : 3 , "orange" : 2} , 
                         [{"name" : "aaa" , "inventory" : {"apple" : 1 } } , 
                          {"name" : "bbb" , "inventory" : {"apple" : 1 ,"orange" : 1}},
                         {"name" : "ccc" , "inventory" : {"apple" : 1 ,"orange" : 1}},
                         {"name" : "ddd" , "inventory" : {"apple" : 2 ,"orange" : 1}},
                         {"name" : "eee" , "inventory" : {"apple" : 1 ,"orange" : 1}}]))

T8 :  [{'bbb': {'apple': 1, 'orange': 1}}, {'ddd': {'apple': 2, 'orange': 1}}]


#### Test Case 9 : If the datatype of order is incorrect

In [654]:
print("T9 : " , inventory_Allocator([{"apple":3}], ""))

T9 :  Invalid Order


## Additional Test Runs

In [655]:
print("T10 : " , inventory_Allocator({"apple" : 3 , "orange" : 1} , 
                         [{"name" : "aaa" , "inventory" : {"apple" : 1 } } , 
                            {"name" : "bbb" , "inventory" : {"banana" : 1,"apple" : 1}},
                          {"name" : "ccc" , "inventory" : {"orange" : 3 } } ,
                          {"name" : "ddd" , "inventory" : {"apple" : 2,"orange" : 3 } }
                         ]))

T10 :  [{'aaa': {'apple': 1}}, {'ddd': {'apple': 2, 'orange': 1}}]


In [656]:
print("T11 : " , inventory_Allocator({"apple" : 1 , "orange" : 1} , 
                         [{"name" : "aaa" , "inventory" : {"apple" : 1 } } , 
                          {"name" : "bbb" , "inventory" : {"orange" : 1 } } ,
                          {"name" : "ccc" , "inventory" : {"orange" : 1}}]))

T11 :  [{'aaa': {'apple': 1}}, {'bbb': {'orange': 1}}]


In [657]:
print("T12 : " , inventory_Allocator({ "apple" : 2}, [{ "name": "owd", "inventory": { "apple": 1 }} ]))

T12 :  []


In [658]:
print("T13 : " , inventory_Allocator({ "apple" : 1 , "orange" : 1 },
                                     [{ "name": "owd", "inventory": { "apple": 0 }} , 
                                      { "name": "abc", "inventory": { "orange": 1 } }]))

T13 :  []


In [659]:
print("T13 : " , inventory_Allocator({ "apple" : 5 , "banana" : 5 },
                          [{ "name": "aaa", "inventory": { "orange": 5, "apple" : 1 }} ,
                           { "name": "bbb", "inventory": { "orange": 3 } },
                          { "name": "ccc", "inventory": { "orange": 3, "banana" : 3 } },
                          { "name": "ddd", "inventory": { "apple": 4, "banana": 2 } },
                          { "name": "abc", "inventory": { "orange": 3 } }]))

T13 :  [{'aaa': {'apple': 1}}, {'ccc': {'banana': 3}}, {'ddd': {'apple': 4, 'banana': 2}}]


In [660]:
print("T14 : " , inventory_Allocator({ "apple" : 2}, ""))

T14 :  []
